<a href="https://colab.research.google.com/github/zricky2/unc_comp590CV/blob/master/CompletedA6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!wget "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
#!unzip tiny-imagenet-200.zip

# Custom dataset


In [0]:
import os
import torch
import torchvision
from torch.utils.data import Dataset

from PIL import Image


def pil_loader(image_path):
    with open(image_path, 'rb') as f:
        return Image.open(f).convert('RGB')


class CustomTrainDataset(Dataset):

    def __init__(self, data_dir, transform=None):
        """
        Args:
            data_dir: the dataset folder. The folder should be structured as:
                label1/image/500 .JPEG images
                label2/image/500 .JPEG images
                ...
                label200/image/500 .JPEG images
        """
        #“prefix/tiny-imagenet-200/train”
        self.data_dir = data_dir
        self.transform = transform
        self.samples = []#(image_path, class_id)
        self.classes = []

        self.root = os.path.normpath(data_dir)
        sortedList = sorted(os.listdir(data_dir))#return sorted list of entries in directory
        for class_id, class_name in enumerate(sortedList):
            self.classes.append(class_name)
            image_path = os.path.join(self.root, class_name)
            image_path = os.path.join(image_path, "images")
            for images in os.listdir(image_path): #root/class/images
                self.samples.append((os.path.join(image_path, images), class_id))
                


    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index): #return (image_path, class_id)
        """Don't forget to apply the transform before returning."""
        image_path, class_id = self.samples[index]
        sample = pil_loader(image_path)
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, class_id
        

class CustomValDataset(Dataset):
    """This is the Validation Dataset.

    Use this as a reference to implement you CustomTrainDataset.
    Remember because the training data and validation data are structured differently,
    you shouldn't directly use the code below. You should design it according to the
    training data folder's structure.
    """
    def __init__(self, data_dir, transform=None):
        self.root = os.path.normpath(data_dir)
        self.transform = transform

        classes_file = os.path.join(self.root, '../wnids.txt')

        self.classes = []
        with open(classes_file, 'r') as f:
            self.classes = f.readlines()
            for i in range(len(self.classes)):
                self.classes[i] = self.classes[i].strip()
        self.classes.sort()

        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

        images_y_file = os.path.join(self.root, 'val_annotations.txt')
        with open(images_y_file, 'r') as f:
            lines = f.readlines()

        self.class_dict = {}
        for line in lines:
            cols = line.split('\t')
            if len(cols) < 2:
                continue
            img_filename = cols[0].strip()
            img_class = cols[1].strip()
            self.class_dict[img_filename] = img_class

        self.samples = []
        images_dir = os.path.join(self.root, 'images')
        for _root, _dirs, files in sorted(os.walk(images_dir)):
            for image_name in files:
                image_path = os.path.join(_root, image_name)
                c = self.class_dict[image_name]
                idx = self.class_to_idx[c]
                self.samples.append((image_path, idx))

    def __getitem__(self, i):
        path, target = self.samples[i]
        sample = pil_loader(path)
        if self.transform is not None:
            sample = self.transform(sample)

        return sample, target

    def __len__(self):
        return len(self.samples)

# New Section

# Custom Alexnet

In [0]:
import torch
import torch.nn as nn

class CustomAlexnet(nn.Module):
    """Alexnet implementation for comp 590.
    
    Remember, you are required to implement alexnet using
    nn.Conv2d instead of nn.Linear.
    Failing to do that will lead to 0 points for this task.
    """
    def __init__(self, num_classes=200):
        super(CustomAlexnet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=8, stride=2, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        #self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Conv2d(256, 4096, kernel_size=6, stride=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Conv2d(4096, 4096, kernel_size=1, stride=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(4096, 200, kernel_size=1, stride=3, padding=1),
        )


    def forward(self, x):
        x = self.features(x)
        #x = self.avgpool(x)
        
        x = self.classifier(x)
        x = torch.flatten(x, 1)
        return x

# Helper functions

In [0]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

# Train and validation functions

In [0]:
import os
import random
import shutil
import time
import warnings

def train(train_loader, model, criterion, optimizer, epoch, dev):

    print_frequency = 10
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        images = images.to(dev)
        target = target.to(dev)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_frequency == 0:
            progress.display(i)
        
    return losses.avg


def validate(val_loader, model, criterion, dev):

    print_frequency = 10
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            images = images.to(dev)
            target = target.to(dev)            

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_frequency == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg, top5.avg, losses.avg


# Main block


In [0]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torchvision.transforms as transforms
from torchsummary import summary


if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"


model = CustomAlexnet(200)

datadir = './tiny-imagenet-200'

model.to(dev)

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().to(dev)

lr = 0.001
momentum = 0.9
weight_decay = 1e-5
optimizer = torch.optim.SGD(model.parameters(), lr=lr, #SGD
                            momentum=momentum,
                            weight_decay=weight_decay)

# Data loading code
traindir = os.path.join(datadir, 'train')
valdir = os.path.join(datadir, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])

print('=> using CustomTrainDataset')
train_dataset = CustomTrainDataset(
    traindir,
    transforms.Compose([
        # transforms.RandomResizedCrop(64),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
print('=> using CustomValDataset')
val_dataset = CustomValDataset(
    valdir,
    transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

# The following 3 lines are for verification of your model structure
# They should output exactly the same lines as the picture in the instruction
print('For 3x64x64 images, the model will generate outputs in the following shapes:')
summary(model, (3, 64, 64))
assert train_dataset.classes == val_dataset.classes

# Modify batch size to see the difference
batch_size = 256 #64-256
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=4, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=True)

best_acc1 = 0
epochs= 20
for epoch in range(epochs):

    # train for one epoch
    train_loss = train(train_loader, model, criterion, optimizer, epoch, dev)

    # evaluate on validation set
    acc1, acc5, val_loss = validate(val_loader, model, criterion, dev)
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

    # Don't forget to include model_best.pth.tar in your report
    save_checkpoint({
        'epoch': epoch + 1,
        #'arch': getpass.getuser(),
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)



=> using CustomTrainDataset
=> using CustomValDataset
For 3x64x64 images, the model will generate outputs in the following shapes:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 31, 31]          12,352
              ReLU-2           [-1, 64, 31, 31]               0
         MaxPool2d-3           [-1, 64, 29, 29]               0
            Conv2d-4          [-1, 192, 29, 29]         307,392
              ReLU-5          [-1, 192, 29, 29]               0
         MaxPool2d-6          [-1, 192, 14, 14]               0
            Conv2d-7          [-1, 384, 14, 14]         663,936
              ReLU-8          [-1, 384, 14, 14]               0
            Conv2d-9          [-1, 256, 14, 14]         884,992
             ReLU-10          [-1, 256, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         590,080
             ReLU-12          [-1, 2